In [1]:
# TRANSFER LEARNING FINE TUNING -  MASS CALC CLASSIFIER USING MAMMOGRAM PATCHES IN BCDR
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
%matplotlib inline
tensorflow.keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU
import time


vid='v003_2'

In [2]:
# Create CNN

IMG_SIZE=224

from tensorflow.keras.applications.xception import Xception
base_model=Xception(weights='imagenet',include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512)(x)
x=ReLU()(x)
x=Dropout(0.5)(x)
x=Dense(512)(x) 
x=ReLU()(x)
preds=Dense(1,activation='sigmoid')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

 block4_sepconv1_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 28, 28, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 28, 28, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 14, 14, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 14, 14, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 14, 14, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 block10_sepconv1_act (Activati  (None, 14, 14, 728)  0          ['add_7[0][0]']                  
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 14, 14, 728)  536536     ['block10_sepconv1_act[0][0]']   
 v2D)     

                                                                                                  
 block12_sepconv3_act (Activati  (None, 14, 14, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 14, 14, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 14, 14, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_10 (A

In [4]:
len(model.layers)

139

In [5]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy']) 

In [6]:
# Image preprocessing and data augmentation

batch_size=8

train_datagen = ImageDataGenerator(rescale=1./255,
                         horizontal_flip = True,
                         vertical_flip = True,
                         fill_mode="wrap",
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         zoom_range = 0.2,
                         rotation_range = 90,
                         shear_range=0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('mass_ben_mal_class02/train0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class02/valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)


Found 537 images belonging to 2 classes.
Found 129 images belonging to 2 classes.


In [7]:
#Callbacks
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [8]:
#Training stage 1
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "mass_ben_mal_class_xception_weights_"+vid+".h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=3,
                   verbose=1,
                   callbacks = callbacks)  

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8664\1152529500.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/3
67/67 [==============================] - 59s 804ms/step - loss: 0.6023 - accuracy: 0.7259 - val_loss: 0.5559 - val_accuracy: 0.7891 - lr: 0.0010
Epoch 2/3
67/67 [==============================] - 51s 769ms/step - loss: 0.4707 - accuracy: 0.7902 - val_loss: 0.5342 - val_accuracy: 0.7969 - lr: 0.0010
Epoch 3/3
67/67 [==============================] - 50s 745ms/step - loss: 0.4709 - accuracy: 0.7826 - val_loss: 0.5382 - val_accuracy: 0.7812 - lr: 0.0010


In [9]:
#Training stage 2
for layer in model.layers[:90]:
    layer.trainable=False
for layer in model.layers[90:]:  #last ~33% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=10,
                   verbose=1,
                   callbacks = callbacks)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8664\81111685.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/10
67/67 [==============================] - 101s 1s/step - loss: 0.4477 - accuracy: 0.7788 - val_loss: 0.5125 - val_accuracy: 0.8516 - lr: 1.0000e-04
Epoch 2/10
67/67 [==============================] - 92s 1s/step - loss: 0.4213 - accuracy: 0.8412 - val_loss: 0.4121 - val_accuracy: 0.8516 - lr: 1.0000e-04
Epoch 3/10
67/67 [==============================] - 90s 1s/step - loss: 0.3565 - accuracy: 0.8374 - val_loss: 0.4732 - val_accuracy: 0.8438 - lr: 1.0000e-04
Epoch 4/10
67/67 [==============================] - 92s 1s/step - loss: 0.3706 - accuracy: 0.8507 - val_loss: 0.4032 - val_accuracy: 0.8672 - lr: 1.0000e-04
Epoch 5/10
67/67 [==============================] - 91s 1s/step - loss: 0.3760 - accuracy: 0.8582 - val_loss: 0.4155 - val_accuracy: 0.8359 - lr: 1.0000e-04
Epoch 6/10
67/67 [==============================] - 90s 1s/step - loss: 0.3121 - accuracy: 0.8771 - val_loss: 0.4466 - val_accuracy: 0.8438 - lr: 1.0000e-04
Epoch 7/10
67/67 [==============================] - 90s 1

In [10]:
#Training stage 3
#time.sleep(360)
for layer in model.layers:  #All layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
callbacks = get_callbacks(name_weights = name_weights, patience_lr=5)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=150,
                   verbose=1,
                   callbacks = callbacks)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8664\2279829289.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/150
67/67 [==============================] - 232s 3s/step - loss: 0.3821 - accuracy: 0.8488 - val_loss: 0.4195 - val_accuracy: 0.8516 - lr: 1.0000e-05
Epoch 2/150
67/67 [==============================] - 208s 3s/step - loss: 0.3495 - accuracy: 0.8544 - val_loss: 0.4558 - val_accuracy: 0.8516 - lr: 1.0000e-05
Epoch 3/150
67/67 [==============================] - 208s 3s/step - loss: 0.3602 - accuracy: 0.8601 - val_loss: 0.4702 - val_accuracy: 0.8594 - lr: 1.0000e-05
Epoch 4/150
67/67 [==============================] - 211s 3s/step - loss: 0.2934 - accuracy: 0.8677 - val_loss: 0.4889 - val_accuracy: 0.8438 - lr: 1.0000e-05
Epoch 5/150
67/67 [==============================] - 207s 3s/step - loss: 0.3459 - accuracy: 0.8582 - val_loss: 0.4996 - val_accuracy: 0.8281 - lr: 1.0000e-05
Epoch 6/150
67/67 [==============================] - 208s 3s/step - loss: 0.2985 - accuracy: 0.8658 - val_loss: 0.4996 - val_accuracy: 0.8203 - lr: 1.0000e-05
Epoch 7/150
67/67 [===========================

In [11]:
model.metrics_names

['loss', 'accuracy']

In [12]:
# Validation accuracy, validation data confusion matrix, area under the roc score
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class02/valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(valid_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(valid_generator.classes, Y_pred)
prec=precision_score(valid_generator.classes, y_pred,pos_label=1)
rec=recall_score(valid_generator.classes, y_pred,pos_label=1)
spe=recall_score(valid_generator.classes, y_pred,pos_label=0)
f1s=f1_score(valid_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(valid_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))

print('Sensitivity: ', rec)
print('Specificity: ', spe)
print('Precision score:', prec)
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('F1 score:', f1s)
print('MCC score:', mcc)
print('ROC AUC score:', ras)

Found 129 images belonging to 2 classes.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_8664\3759042783.py:13: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(valid_generator)


Confusion Matrix:
[[75 10]
 [ 9 35]]
Sensitivity:  0.7954545454545454
Specificity:  0.8823529411764706
Precision score: 0.7777777777777778
Accuracy: 0.8527131782945736
F1 score: 0.7865168539325843
MCC score: 0.6742116655788051
ROC AUC score: 0.8871657754010694


In [13]:
#Print estimation result for ensemble model
print(Y_pred)
print(valid_generator.classes)

[[5.2049343e-02]
 [8.8215776e-02]
 [3.2232746e-01]
 [1.7334051e-01]
 [3.3146355e-02]
 [5.8250770e-04]
 [3.0353600e-01]
 [3.3115097e-03]
 [3.5901184e-04]
 [4.3408504e-01]
 [9.4198389e-03]
 [5.4963008e-02]
 [2.9042251e-03]
 [1.2301378e-01]
 [4.4855776e-01]
 [4.3467026e-02]
 [1.0778161e-01]
 [2.6908955e-01]
 [1.2342920e-01]
 [2.2528088e-02]
 [2.9856116e-01]
 [1.7971407e-01]
 [6.2871926e-02]
 [2.1113278e-02]
 [1.6952889e-02]
 [1.0052797e-01]
 [2.1047106e-03]
 [5.1750165e-01]
 [1.3467477e-02]
 [2.5005382e-01]
 [3.6577088e-01]
 [6.9504619e-01]
 [1.2084345e-02]
 [5.4481290e-02]
 [2.4728760e-02]
 [1.2112311e-02]
 [1.0389668e-02]
 [5.7355326e-01]
 [8.3823974e-04]
 [5.6357379e-03]
 [9.8782080e-01]
 [8.9422864e-01]
 [2.0570798e-02]
 [9.2947677e-02]
 [1.3351033e-02]
 [7.4539608e-01]
 [9.2881966e-01]
 [3.8234703e-02]
 [5.4384869e-01]
 [3.9826729e-03]
 [4.0215021e-01]
 [1.4261728e-02]
 [2.7561787e-01]
 [4.2554462e-01]
 [3.3842549e-01]
 [2.8085968e-01]
 [1.4530116e-02]
 [7.7281343e-03]
 [1.0818163e-0

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('mass_ben_mal_class02/test', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)


# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(test_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(test_generator.classes, Y_pred)
prec=precision_score(test_generator.classes, y_pred,pos_label=1)
rec=recall_score(test_generator.classes, y_pred,pos_label=1)
spe=recall_score(test_generator.classes, y_pred,pos_label=0)
f1s=f1_score(test_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(test_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))

print('Sensitivity: ', rec)
print('Specificity: ', spe)
print('Precision score:', prec)
print('Accuracy:', accuracy_score(test_generator.classes, y_pred))
print('F1 score:', f1s)
print('MCC score:', mcc)
print('ROC AUC score:', ras)

Found 169 images belonging to 2 classes.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_8664\3439002409.py:15: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_generator)


Confusion Matrix:
[[103  10]
 [ 20  36]]
Sensitivity:  0.6428571428571429
Specificity:  0.911504424778761
Precision score: 0.782608695652174
Accuracy: 0.8224852071005917
F1 score: 0.7058823529411765
MCC score: 0.5862662288318407
ROC AUC score: 0.8337547408343868


In [15]:
#Print estimation result for ensemble model
print(Y_pred)
print(test_generator.classes)

[[3.7212855e-01]
 [3.3212647e-01]
 [2.9750419e-01]
 [3.0484550e-02]
 [1.2527342e-01]
 [2.1154977e-02]
 [7.5046055e-02]
 [3.2486029e-03]
 [2.9478662e-03]
 [8.0043497e-03]
 [6.8416423e-03]
 [5.7806796e-03]
 [1.6273259e-01]
 [1.0985115e-02]
 [5.1092088e-02]
 [3.6300858e-04]
 [4.7975519e-01]
 [5.4425049e-01]
 [5.4055103e-04]
 [1.7378825e-01]
 [4.8161322e-01]
 [2.3623606e-02]
 [8.7222122e-02]
 [7.5016397e-01]
 [1.5961584e-03]
 [5.4902043e-03]
 [7.8116253e-02]
 [6.2945521e-01]
 [5.9071374e-01]
 [1.0472776e-01]
 [1.2728071e-02]
 [3.1289852e-01]
 [1.7581463e-02]
 [4.2769381e-01]
 [7.9740487e-02]
 [4.7684335e-03]
 [9.7650271e-03]
 [4.4154122e-01]
 [2.8818980e-02]
 [5.5836111e-01]
 [4.6494640e-02]
 [1.0380405e-02]
 [3.6506739e-02]
 [1.9264402e-02]
 [3.4471878e-01]
 [7.5210100e-03]
 [6.2732040e-03]
 [1.0395944e-02]
 [6.5177250e-01]
 [4.3961547e-02]
 [8.4317587e-03]
 [4.5219705e-01]
 [3.8992543e-02]
 [1.9179627e-03]
 [2.5789093e-03]
 [3.2137063e-01]
 [3.9377734e-01]
 [4.1617641e-01]
 [4.5219675e-0